In [9]:
#This code is for the 2nd question of the assessment for the Energy Infrastructure Module at the University of Oxford.
#This code runs a powerflow on a proposed grid for 200 houses in the Philippines
#This power flow should be run using a normal Python 3 kernal
#The following code was made by the Philippines Team and edited by Canidate 1056380

In [10]:
#Importing pandapower and setting up our demand specs
import pandapower as pp
''' Define demand of each building (MW) '''

p_house =0.5e-3                    # house
p_shop =0.4e-3                      # shop 
p_school =7e-3                     # school 
q_demand=0                          # have not included reactive power 

# Buildings have been grouped in 3's to balance load of 3 phases.

p_house_group = 3*p_house           # 3 houses 
p_shop_group = p_shop+2*p_house     # 1 shop and 2 houses. 



''' Define volatge at each level of grid (kV) '''

Cadiz_Substation_Voltage = 69
Local_Substation_Voltage = 10
District_Transformer_Voltage = 0.415 



''' Define length of lines (km) '''

Cadiz_Local_Length = 25                   # Cadiz to local substation
Local_District_Length =0.06               # Local substation to housing district transformers
Local_District_Diagonal_Length = 0.0849    # Local substation to school and villa transformers
District_School_Length = 0.0778            # School district transformer to school
Local_Lines_Length=((4*Local_District_Length)+(2*Local_District_Diagonal_Length))*1000 # Total Length of local lines in m

# Define matrix with distances from housing district transformer to each house group

Road_width = 0.1                          # width of road              
House_length = 0.1                        # length of house
L_primary=0.035
#km, distance from district transformer to start of every block
half_length=0.05
full_length=0.1

line_length =[L_primary+2*half_length+full_length,L_primary+2*half_length+2*full_length,L_primary+2*half_length+4*full_length,L_primary+2*half_length+5*full_length]
Houses_Line_Length_Total=16*(L_primary+2*half_length+full_length+ L_primary+2*half_length+2*full_length+ L_primary+2*half_length+4*full_length+ L_primary+2*half_length+5*full_length)

# Villa District transformer to villa housing groups

L_primary_villa=0.015
villa_line_length=[L_primary_villa+half_length+2*full_length,L_primary_villa+3*full_length+half_length,L_primary_villa+half_length+5*full_length, L_primary_villa+6*full_length+half_length]
Villas_Line_Length_Total=4*(L_primary_villa+half_length+2*full_length+ L_primary_villa+3*full_length+half_length + L_primary_villa+half_length+5*full_length + L_primary_villa+6*full_length+half_length)


District_Line_Total_Length=(Houses_Line_Length_Total+Villas_Line_Length_Total)*1000

''' Define cable properties - resistance (ohms/km), reactance (ohms/km), current cpacity (kA) '''

#For 69kV cables, size 150mm2 from Eland 2XS(F)2Y, A2XS(F)2Y
r_cadiz_local_150_al= 0.206
max_i_cadiz_local_150_al_air_flatspaced=431e-3 
x_cadiz_local_150_al= 0.10

#For 33kV cables, size 50mm2, from Eland BS 6622 XLPE/PVC Medium Voltage
#Note at the moment the code uses this 33kV data for the 69kV cables, as the 69kV data gives very low loading. 
r_cadiz_local_50_al= 0.350
max_i_cadiz_local_50_al_air_flatspaced=220e-3
x_cadiz_local_50_al= 0.10

#For 10 kV cables, from Cleveland N2XXS2Y 10KV POWER CABLE SPECIFICATION SHEET
r_Local_District_35=1.1
x_Local_District_35=0.24
r_Local_District_50=1.6
x_Local_District_50=0.24

#For 0.415 kV cables
#size 2.5mm2 from Eland BS 5467
r_district_house_025 = 7.41
max_i_district_house_025=28e-3
x_district_house_025_al= 0.10

#4mm2 cables not used in this powerflow, but specs were considered
#4mm2
#r_district_house_4= 4.61
#max_i_district_house_4=36e-3
#x_district_house_4_al= 0.10

#6mm2
r_district_house_6= 3.08
max_i_district_house_6=44e-3
x_district_house_6_al= 0.10

#4mm2 cables not used in this powerflow, but specs were considered
#10mm2
#r_district_house_10= 1.83
#max_i_district_house_10=58e-3
#x_district_house_10_al= 0.10



In [11]:
''' Create network '''

net = pp.create_empty_network()


''' Substations '''

# Define caidz substation bus, set as external grid
Cadiz_Substation= pp.create_bus(net, vn_kv =Cadiz_Substation_Voltage, name = "Cadiz Substation")
pp.create_ext_grid(net, bus=Cadiz_Substation, vm_pu=1.02, name="Swing bus")

# Define local substation bus, the Cadiz (69kV) to Local (10kV) transformer, and line 
Local_Substation= pp.create_bus(net, vn_kv =Local_Substation_Voltage, name = "Local Substation")
pp.create_line_from_parameters(net,from_bus=Cadiz_Substation, to_bus=Local_Substation ,length_km=Cadiz_Local_Length, r_ohm_per_km =r_cadiz_local_50_al, x_ohm_per_km =x_cadiz_local_50_al,c_nf_per_km = 1e-9, max_i_ka=max_i_cadiz_local_50_al_air_flatspaced,name = "CadizSubstation-LocalSubstation")
pp.create_transformer_from_parameters(net, hv_bus=Cadiz_Substation, lv_bus=Local_Substation, sn_mva=40, vn_hv_kv=Cadiz_Substation_Voltage , vn_lv_kv=Local_Substation_Voltage, vkr_percent=0.34, vk_percent=16.2, pfe_kw=18.0, i0_percent=0.05, name="LocalTransformer")


''' School '''

# Define school district bus, the local (10kV) to school district (0.415kV) transformer, and line  
School_district_bus = pp.create_bus(net, vn_kv = District_Transformer_Voltage, name = "School Bus")
pp.create_transformer_from_parameters(net, hv_bus=Local_Substation, lv_bus=School_district_bus ,sn_mva=0.4, vn_hv_kv=Local_Substation_Voltage , vn_lv_kv=District_Transformer_Voltage, vkr_percent=1.325, vk_percent=4, pfe_kw=0.95, i0_percent=0.2375, name="School Transformer")
pp.create_line_from_parameters(net,from_bus=Local_Substation, to_bus=School_district_bus ,length_km=Local_District_Diagonal_Length, r_ohm_per_km =r_Local_District_50, x_ohm_per_km =x_Local_District_50  ,c_nf_per_km = 1e-9, max_i_ka=300e-3,name = "LocalSubstation to School District")

# Define school bus, set as a load, creat lines from school district bus to school bus
School_bus = pp.create_bus(net, vn_kv = District_Transformer_Voltage, name = "School Bus")
pp.create_load(net, bus=School_bus, p_mw=p_school, q_mvar=q_demand, name='School')
pp.create_line_from_parameters(net,from_bus=School_district_bus, to_bus=School_bus ,length_km=District_School_Length, r_ohm_per_km =r_Local_District_50, x_ohm_per_km =x_Local_District_50  ,c_nf_per_km = 1e-9, max_i_ka=300e-3,name = "School District to School")


''' Villas '''

# Define Villa district bus, the local (10kV) to villa district (0.415kV) transformer, and line  
Villa_district_bus = pp.create_bus(net, vn_kv =District_Transformer_Voltage , name = "Eight Houses District Station")
pp.create_transformer_from_parameters(net, hv_bus=Local_Substation, lv_bus=Villa_district_bus,sn_mva=0.4, vn_hv_kv=Local_Substation_Voltage , vn_lv_kv=District_Transformer_Voltage, vkr_percent=1.325, vk_percent=4, pfe_kw=0.95, i0_percent=0.2375, name="8 Houses Transformer")
pp.create_line_from_parameters(net,from_bus=Local_Substation, to_bus=Villa_district_bus, length_km=Local_District_Diagonal_Length, r_ohm_per_km =r_Local_District_50, x_ohm_per_km =x_Local_District_50  ,c_nf_per_km = 1e-9, max_i_ka=300e-3,name = "LocalSubstation to 8 Houses Station")

# Define villa buses, set as a load, create lines from villa district tbus to villa buses
for villa in range (4):
    villa_group_bus = [None]*4
for x in range(4):
        villa_group_bus[x] = pp.create_bus(net, vn_kv = District_Transformer_Voltage, name = "Villa Group H%s" %(x))
        pp.create_load(net, bus= villa_group_bus[x], p_mw=p_house_group, q_mvar=q_demand, name = "Villa Group H%s" %(x))
        pp.create_line_from_parameters(net,from_bus= Villa_district_bus, to_bus= villa_group_bus[x], length_km=villa_line_length[x], r_ohm_per_km = r_district_house_6, x_ohm_per_km = x_district_house_6_al, c_nf_per_km = 1e-9, max_i_ka=max_i_district_house_6, name = "Six Villas District Bus - Villa Group H%s" %(x))


''' Houses '''

# For each of the 4 districts, there are 4 blocks with 4 house groups (each group has 3 houses). 
for district in range(4):
   
    # Define house district buses, the local (10kV) to house district (0.415kV) transformer, and lines 
    District_bus = pp.create_bus(net, vn_kv = District_Transformer_Voltage, name = "District Station D%s" %(district))
    pp.create_transformer_from_parameters(net, hv_bus=Local_Substation, lv_bus=District_bus, sn_mva=0.4, vn_hv_kv=Local_Substation_Voltage , vn_lv_kv=District_Transformer_Voltage, vkr_percent=1.325, vk_percent=4, pfe_kw=0.95, i0_percent=0.2375, name="DistrictTransformer")
    pp.create_line_from_parameters(net,from_bus=Local_Substation, to_bus=District_bus ,length_km=Local_District_Length, r_ohm_per_km =r_Local_District_50, x_ohm_per_km =x_Local_District_50  ,c_nf_per_km = 1e-9, max_i_ka=300e-3,name = "Local Substation-DistrictSubstation")
   
    for block in range(4):
            for i in range (4):
                house_group_bus = [None]*4    
                
                if (i==0 and block == 1): #if function to create a house group with a shop at the start of the second block in each district             
                    house_group_bus[i] = pp.create_bus(net, vn_kv = District_Transformer_Voltage, name = "Shop and House%s B%s H%s" %(district, block, i))
                    pp.create_load(net, bus= house_group_bus[i], p_mw=p_shop_group, q_mvar=q_demand, name = "Shop and house%s B%s H%s" %(district, block, i))
                    pp.create_line_from_parameters(net,from_bus= District_bus, to_bus= house_group_bus[i],length_km=line_length[i], r_ohm_per_km = r_district_house_6, x_ohm_per_km = x_district_house_6_al,  c_nf_per_km = 1e-9, max_i_ka=max_i_district_house_6,  name = "District Bus - Shop Group d%s B%s H%s" %(district, block, i))
                
                else: 
                    house_group_bus[i] = pp.create_bus(net, vn_kv = District_Transformer_Voltage, name = "D%s B%s House Group%s" %(district, block, i))
                    pp.create_load(net, bus= house_group_bus[i], p_mw=p_house_group, q_mvar=q_demand, name = "D%s B%s House Group%s" %(district, block, i))
                    pp.create_line_from_parameters(net,from_bus= District_bus, to_bus= house_group_bus[i], length_km=line_length[i], r_ohm_per_km = r_district_house_6, x_ohm_per_km = x_district_house_6_al,   c_nf_per_km = 1e-9, max_i_ka=max_i_district_house_6, name = "District Bus - House Group_bus D%s B%s H%s" %(district, block, i))
    



In [6]:
''' Exporting the network that has been set-up to excel'''

#Other data can be found by deleting the hashtag next to these to excel lines of code

#net.load.to_excel('Loads.xlsx')
#net.trafo.to_excel('Transformers.xlsx')
net.line.to_excel('LinesPhi.xlsx')
#net.bus.to_excel('Buses.xlsx')



''' Solving the network and exporting results to excel '''

pp.runpp(net, algorithm='nr')

#net.res_bus.to_excel('Bus Results.xlsx')
net.res_line.to_excel('Line Results.xlsx')
#net.res_trafo.to_excel('Transformer Results.xlsx')


In [7]:
#Determining local line lengths for cost calculations
Local_Lines_Length

409.8

In [8]:
#Determining district line lengths for cost calculations
District_Line_Total_Length

35280.0